In [53]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import pyarrow
import fastparquet
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import numpy as np
import joblib
from surprise import Dataset, Reader, SVD, SVDpp, KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV
pd.set_option('display.max_colwidth', None)

In [3]:
def connect():
    conn = None
    try:
        print('Connecting...')
        conn = psycopg2.connect(
            host='10.10.248.108',
            database='postgres',
            user='eatin',
            password='eatin')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    print('Connection Successful')

    return conn


def execute_select(conn, query, column_names):
    cursor = conn.cursor()
    data = pd.DataFrame()
    try:
        cursor.execute(query)
        data = cursor.fetchall()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    cursor.close()

    return pd.DataFrame(data, columns=column_names)


def load_ratings_from_pg():
    query = "select * from ratings;"
    conn = connect()
    df = execute_select(conn, query, ['user_id', 'recipe_index', 'rating'])
    conn.close()
    return df

In [26]:
# Load the data into a pandas dataframe
df = load_ratings_from_pg()

# Define the Reader object
reader = Reader(rating_scale=(0.5, 5))

# Load the data into the surprise dataset format
data = Dataset.load_from_df(df[['user_id', 'recipe_index', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.25)

# Define the algorithm to use
algo = SVD()

# Train the algorithm on the training set
algo.fit(trainset)

# Test the algorithm on the test set
predictions = algo.test(testset)

# Evaluate the accuracy of the algorithm
accuracy.rmse(predictions)


# Use the algorithm to predict the user's top 10 recipes
user_id = '8DZDbeh0aWXKbvyvahpIl24kNEs2'
items_to_predict = [i for i in range(1, len(df['recipe_index'].unique()) + 1)]
user_ratings = [(user_id, str(item), 0) for item in items_to_predict]
user_predictions = algo.test(user_ratings)
top_n = sorted(user_predictions, key=lambda x: x.est, reverse=True)[:10]
for pred in top_n:
    print("Recipe {}, predicted rating: {}".format(pred.iid, pred.est))

Connecting...
Connection Successful
RMSE: 1.2911
Recipe 1, predicted rating: 3.876383568568444
Recipe 2, predicted rating: 3.876383568568444
Recipe 3, predicted rating: 3.876383568568444
Recipe 4, predicted rating: 3.876383568568444
Recipe 5, predicted rating: 3.876383568568444
Recipe 6, predicted rating: 3.876383568568444
Recipe 7, predicted rating: 3.876383568568444
Recipe 8, predicted rating: 3.876383568568444
Recipe 9, predicted rating: 3.876383568568444


In [28]:
user_predictions

[Prediction(uid='8DZDbeh0aWXKbvyvahpIl24kNEs2', iid='1', r_ui=0, est=3.876383568568444, details={'was_impossible': False}),
 Prediction(uid='8DZDbeh0aWXKbvyvahpIl24kNEs2', iid='2', r_ui=0, est=3.876383568568444, details={'was_impossible': False}),
 Prediction(uid='8DZDbeh0aWXKbvyvahpIl24kNEs2', iid='3', r_ui=0, est=3.876383568568444, details={'was_impossible': False}),
 Prediction(uid='8DZDbeh0aWXKbvyvahpIl24kNEs2', iid='4', r_ui=0, est=3.876383568568444, details={'was_impossible': False}),
 Prediction(uid='8DZDbeh0aWXKbvyvahpIl24kNEs2', iid='5', r_ui=0, est=3.876383568568444, details={'was_impossible': False}),
 Prediction(uid='8DZDbeh0aWXKbvyvahpIl24kNEs2', iid='6', r_ui=0, est=3.876383568568444, details={'was_impossible': False}),
 Prediction(uid='8DZDbeh0aWXKbvyvahpIl24kNEs2', iid='7', r_ui=0, est=3.876383568568444, details={'was_impossible': False}),
 Prediction(uid='8DZDbeh0aWXKbvyvahpIl24kNEs2', iid='8', r_ui=0, est=3.876383568568444, details={'was_impossible': False}),
 Predict

In [27]:
df['recipe_index'].unique()

array([1411, 7710, 5567, 7928, 2541, 7557, 7063,  982, 5500])

In [52]:
# load data
ratings_df = load_ratings_from_pg()
recipes_df = pd.read_parquet('../dataset/recipes.parquet.gzip').reset_index()

# create a mapping for book ids and book names
id_name_map = dict(zip(recipes_df['index'], recipes_df['recipe_title']))

# define a reader object
reader = Reader(rating_scale=(0.5, 5))

# create the dataset
data = Dataset.load_from_df(ratings_df[['user_id', 'recipe_index', 'rating']], reader)

# build the trainset
trainset = data.build_full_trainset()

# define the algorithm
algo = SVD()

# train the algorithm
algo.fit(trainset)

# get user id for recommendations
user_id = '8DZDbeh0aWXKbvyvahpIl24kNEs2'

# get book ids for books not rated by user
recipes_not_rated_by_user = ratings_df[~ratings_df['recipe_index'].isin(ratings_df[ratings_df['user_id']==user_id]['recipe_index'])]['recipe_index']

# make predictions for the books not rated by user
recipe_predictions = []
for recipe_id in recipes_not_rated_by_user.unique():
    predicted_rating = algo.predict(user_id, recipe_id).est
    recipe_predictions.append((recipe_id, predicted_rating))

# sort book predictions by predicted rating in descending order
recipe_predictions.sort(key=lambda x: x[1], reverse=True)

# recommend top 10 books with highest predicted rating
print(f"Top 10 recommended books for user {user_id}:")
for recipe, predicted_rating in recipe_predictions[:10]:
    print(f"recipe:{recipe}, rating:{predicted_rating}, title:{id_name_map[recipe]}")


Connecting...
Connection Successful
Top 10 recommended books for user 8DZDbeh0aWXKbvyvahpIl24kNEs2:
recipe:7647, rating:3.501832786835636, title:Chicken Spaghetti in Tomato Basil Sauce Recipe
recipe:2541, rating:3.334762595500468, title:Chicken In White Gravy Recipe
recipe:1411, rating:3.3019490596848433, title:Chicken Momos Recipe -Delicious Steamed Chicken Dumplings
recipe:5567, rating:3.2752732268423754, title:Bengali Bhapa Doi Recipe
recipe:7557, rating:3.1250445841454657, title:Spaghetti Pasta Alla Puttanesca Recipe
recipe:250, rating:3.122203871574102, title:Andhra Pepper Chicken Recipe - Dry Restaurant Style Pepper Chicken
recipe:7954, rating:3.068307933493099, title:Baked Gobi Manchurian Recipe - Healthy Cauliflower Manchurian Recipe


In [75]:
# load data
ratings_df = load_ratings_from_pg()
recipes_df = pd.read_parquet('../dataset/recipes.parquet.gzip').reset_index()

# create a mapping for book ids and book names
id_name_map = dict(zip(recipes_df['index'], recipes_df['recipe_title']))

# define a reader object
reader = Reader(rating_scale=(0.5, 5))

# create the dataset
data = Dataset.load_from_df(ratings_df[['user_id', 'recipe_index', 'rating']], reader)

# build the trainset
trainset = data.build_full_trainset()

# define the algorithm
algo = SVD()

# train the algorithm
algo.fit(trainset)

# get user id for recommendations
user_id = '8DZDbeh0aWXKbvyvahpIl24kNEs2'

# get book ids for books not rated by user
recipes_not_rated_by_user = ratings_df[~ratings_df['recipe_index'].isin(ratings_df[ratings_df['user_id']==user_id]['recipe_index'])]['recipe_index']

# make predictions for the books not rated by user
recipe_predictions = []
for recipe_id in recipes_not_rated_by_user.unique():
    predicted_rating = algo.predict(user_id, recipe_id).est
    recipe_predictions.append((recipe_id, predicted_rating, recipes_df[recipes_df['index'] == recipe_id]))

# sort book predictions by predicted rating in descending order
recipe_predictions.sort(key=lambda x: x[1], reverse=True)

# recommend top 10 books with highest predicted rating
print(f"Top 10 recommended books for user {user_id}:")
for recipe, predicted_rating, kaki in recipe_predictions[:10]:
    print(f"recipe:{recipe}, rating:{predicted_rating}, title:{id_name_map[recipe]}")


Connecting...
Connection Successful
Top 10 recommended books for user 8DZDbeh0aWXKbvyvahpIl24kNEs2:
recipe:5567, rating:3.526905291568052, title:Bengali Bhapa Doi Recipe
recipe:1411, rating:3.523041449474735, title:Chicken Momos Recipe -Delicious Steamed Chicken Dumplings
recipe:7647, rating:3.5220300193920018, title:Chicken Spaghetti in Tomato Basil Sauce Recipe
recipe:2541, rating:3.3352986283129282, title:Chicken In White Gravy Recipe
recipe:7954, rating:3.322305871590233, title:Baked Gobi Manchurian Recipe - Healthy Cauliflower Manchurian Recipe
recipe:7557, rating:3.2877164292595142, title:Spaghetti Pasta Alla Puttanesca Recipe
recipe:250, rating:2.835469068602279, title:Andhra Pepper Chicken Recipe - Dry Restaurant Style Pepper Chicken


In [78]:
recipe_predictions[1][2]

,index,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category,image,difficulty,total_time
1365,1411,Chicken Momos Recipe -Delicious Steamed Chicken Dumplings,https://www.archanaskitchen.com/chicken-momos-recipe-delicious-steamed-chicken-dumplings,good,51675,4.521562,"Chicken Momos Recipe is a North Eastern recipe and very popular in Tibet as well. Chicken mince flavoured with soy sauce, chilies, pepper are stuffed inside a thin flour dough filling and then steamed.Since this recipe is a steamed dim sum recipe, it is a healthy high protein recipe with tender chicken. Serve Chicken Momos Recipe (Delicious Steamed Chicken Dumplings) along with Peanut Chilli Dipping Sauce for an evening snack.",North East India Recipes,Snack,High Protein Non Vegetarian,30,30,"[2 cups All Purpose Flour (Maida), Salt, to taste, 1-1/2 teaspoon Baking powder, 1 cup Chicken, boiled and minced, 1/2 cup Green peas (Matar), boiled, 1/2 cup Onions, finely chopped, 1 tablespoon Garlic, finely chopped, 1/2 tablespoon Ginger, finely chopped, 1/2 tablespoon Soy sauce, Salt, to taste, 1 Fresh Red chillies, or green chilli, 1/4 tablespoon Whole Black Peppercorns, crushed]","[To prepare Chicken Momos Recipe], prep up with all the ingredients mentioned. Boil the chicken, mince it., Boil the green peas and keep it ready. Grate garlic and ginger or finely chop them., In a mixing bowl, add the refined flour, salt, baking powder and water; and when the flour looks crumbly, knead to a stiff dough, with water and keep aside for about 20 minutes, covered with a wet cloth, so that the dough doesn't dry out., Heat a tablespoon of oil in a wok and add the chopped green chilli or red chilli, onions, chopped ginger and garlic., Saute for about 4-5 minutes and add the boiled chicken mince and boiled peas and mix well till combined, Take it off the heat and mix in the soy sauce, salt and black pepper. Be careful while adding salt since the soy sauce also has salt in it., Divide the dough into equal small portions. Shape them into balls and roll them out thinly, to a circular shape., You can also make moms using a moulds, which will make the process even and easier., Take a dough disk, wet its edges and place a spoonful of filling of chicken, peas and sauteed veggies in the center, bring the edges together to cover the filling, twist carefully to seal the momos while you work on rolling up the edges artfully for the chicken momos. Repeat the process till all the dough and the filling is used up., Meanwhile, get a dimsum steamer or dhokla-idli steamer ready and fill it with some water and keep it for boiling. Arrange the momos on the steamer and steam for about 15-20 minutes or till the momos are steamed well., Serve Chicken Momos Recipe (Delicious Steamed Chicken Dumplings) along with Peanut Chilli Dipping for an evening snack.]",Archana's Kitchen,"[Winter Recipes, Monsoon Recipes, Street food recipes, Monsoon & Rainy Day Recipes, Dumpling Recipes (Momo Recipes), Indo Chinese Recipes (Indian Chinese)]",Indian Snack Recipes,https://www.archanaskitchen.com/images/archanaskitchen/1-Author/shaikh.khalid7-gmail.com/Chicken_Momos_Recipe_Delicious_Steamed_Chicken_Dumplings.jpg,Easy,60


In [58]:
recipes_df[recipes_df['index'] == 7954]

,index,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category,image,difficulty,total_time
7631,7954,Baked Gobi Manchurian Recipe - Healthy Cauliflower Manchurian Recipe,https://www.archanaskitchen.com/baked-gobi-manchurian-recipe-healthy-cauliflower-manchurian-recipe,good,22483,4.409521,"The Baked Gobi Manchurian Recipe is a wholesome and lip smacking recipe that is made from just 1 teaspoon oil. It's packed with flavors from ginger, onion, garlic & chilli sauce. Gobi Manchurian is a very popular Indian Chinese (also known as Indo-Chinese) dish. Typically Gobi manchurian is deep fried and becomes unhealthy to consume it often. But with this recipe, since it is steamed and baked and uses just 1 teaspoon oil, it becomes absolutely ok to enjoy. Serve the Baked Gobi Manchurian Recipe pricked with toothpicks on top of each floret and serve it as an appetizer for your parties along with many other Vegetarian Hakka Noodles.",Indo Chinese,Appetizer,Vegetarian,20,20,"[1 Cauliflower (gobi), cut into florets of lemon size, 1/2 cup Spring Onion Greens, chopped, 6 cloves Garlic, finely chopped, 2 Green Chillies, finely chopped, 2 inch Ginger, finely chopped, 2 tablespoons Corn flour, 1/2 cup Water, or vegetable stock, 1/2 cup Homemade tomato puree, 1 tablespoons Soy sauce, 1 tablespoon Tomato Ketchup, 1 tablespoon Red Chilli sauce, (asian style), 1 teaspoon Oil, 1 tablespoon Oil]","[To begin making the Baked Gobi Manchurian Recipe we will first steam the cauliflower in the steamer for 3 to 4 minutes on high heat., Prepare the steamer with water below, add the gobi into the steamer and steam on high heat for 3 to 4 minutes. The gobi should be just half cooked., Once done, remove it from the steamer and keep aside., The next step is to make the Gobi Manchurian Sauce, Into the mixing bowl, add the corn flour, soya sauce, ketchup, chilli sauce, tomato puree and water. Mix well to combine., Heat oil in a wok or frying pan on high heat; add garlic, ginger, green chillies, spring onions and stir fry for a few seconds., Add the corn flour mixture to the above and cook for a few minutes until thick and the corn flour is cooked. Turn off heat., Toss the partially steamed Cauliflower with the Manchurian Sauce and toss well to combine. Place the Gobi Manchurian in a Baking Tray, to set it for baking., Preheat the oven to 200 C for 10 to 12 minutes. Once the preheated, place the Gobi Manchurian into the oven and make on high for 12 to 15 minutes. You will notice the manchurian will start getting crisp and the sauce will start sticking the Gobi., Once done, remove the Baked Gobi Manchurian from the oven, transfer it to a serving plate., Garnish with spring onion leaves and extra green chillies if you like it spicy., Serve the Baked Gobi Manchurian Recipe pricked with toothpicks on top of each floret and serve it as an appetizer for your parties along with many other Vegetarian Hakka Noodles.]",Archana Doshi,"[Party Food Recipes, Cauliflower Recipes (Phool Gobi), Party Starter & Appetizer Recipes, Manchurian Recipes]",Indian Chinese Recipes,https://www.archanaskitchen.com/images/archanaskitchen/0-Archanas-Kitchen-Recipes/2018/Baked_Gobi_Manchurian_Recipe-5.jpg,Easy,40


In [85]:
import json
b = pd.concat([kaki[2] for kaki in recipe_predictions])
json.loads(pd.concat([kaki[2] for kaki in recipe_predictions]).to_json(orient='records'))

[{'index': 5567,
  'recipe_title': 'Bengali Bhapa Doi Recipe',
  'url': 'https://www.archanaskitchen.com/bengali-bhapa-doi-recipe',
  'record_health': 'good',
  'vote_count': 8689,
  'rating': 4.5531128529,
  'description': 'Bhapa Doi or Baked Yogurt is Hung curd or Greek yogurt blended with milk and condensed milk and baked with a water bath in the oven or steamed on the stove top is a must have sweet dish in the Bengali weddings and special occasions like festivals etc. Bhapa Doi can be flavored with saffron, cardamom etc and can be garnished with fresh fruits or chopped nuts. It can also be given a fruity flavour by adding strawberry or pomegranate reduction and topped with molasses. Serve Bhapa Doi to your friends and family during Holi with some snacks like Baked Paneer Bread Pakora and Jhure-Jhure Aloo Bhaja. Some more recipes that you can try are: ',
  'cuisine': 'Bengali Recipes',
  'course': 'Dessert',
  'diet': 'Vegetarian',
  'prep_time': 10,
  'cook_time': 45,
  'ingredient

In [83]:
b

,index,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category,image,difficulty,total_time
5371,5567,Bengali Bhapa Doi Recipe,https://www.archanaskitchen.com/bengali-bhapa-doi-recipe,good,8689,4.553113,"Bhapa Doi or Baked Yogurt is Hung curd or Greek yogurt blended with milk and condensed milk and baked with a water bath in the oven or steamed on the stove top is a must have sweet dish in the Bengali weddings and special occasions like festivals etc. Bhapa Doi can be flavored with saffron, cardamom etc and can be garnished with fresh fruits or chopped nuts. It can also be given a fruity flavour by adding strawberry or pomegranate reduction and topped with molasses. Serve Bhapa Doi to your friends and family during Holi with some snacks like Baked Paneer Bread Pakora and Jhure-Jhure Aloo Bhaja. Some more recipes that you can try are:",Bengali Recipes,Dessert,Vegetarian,10,45,"[1-1/2 cup Hung Curd (Greek Yogurt), 1 cup Milk, 400 grams Condensed Milk, 1 teaspoon Cardamom Powder (Elaichi), 4 tablespoons Pistachios, chopped, Sultana Raisins, few, Strawberries, few]","[To begin with Bhapa Doi, firstly in a large pan whisk hung curd and condensed milk together with an electric hand blender till it is smooth and creamy., Add milk slowly and whisk again till you see the froth appearing., Add cardamom powder and handful of raisins and give it a quick stir again., Preheat the oven at 180 degree C and meanwhile, take 6 ramekins or earthen ware or any oven safe bowl in which you want to bake the curd. You can also bake it in a large baking tray and later cut into pieces., Pour in the curd mix into the ramekins till the rim., In a baking tray or a drip tray of the oven, place the filled ramekins and pour water, in the tray till the ramekins are half immersed., Place the tray in the oven and bake for 40-45 mins or till the toothpick inserted comes out clean. The baking tie may vary from oven to oven., Take out the tray and let it cool. Once the ramekins are cool completely, garnish with chopped pistachios, strawberries or any fruit or nuts., Refrigerate the Bhapa Doi for minimum 2 hours before use and serve it chilled., Serve Bhapa Doi to your friends and family during Holi with some snacks like Baked Paneer Bread Pakora and Jhure-Jhure Aloo Bhaja.]",Shaheen Ali,"[Festival Recipes, Dahi Recipes (Curd/Yogurt), Strawberry Recipes]",Sweet Recipes (Indian Mithai / Indian Dessert),https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Shaheen_Ali/Bengali_Bhapa_Doi_Recipe-1.jpg,Easy,55
1365,1411,Chicken Momos Recipe -Delicious Steamed Chicken Dumplings,https://www.archanaskitchen.com/chicken-momos-recipe-delicious-steamed-chicken-dumplings,good,51675,4.521562,"Chicken Momos Recipe is a North Eastern recipe and very popular in Tibet as well. Chicken mince flavoured with soy sauce, chilies, pepper are stuffed inside a thin flour dough filling and then steamed.Since this recipe is a steamed dim sum recipe, it is a healthy high protein recipe with tender chicken. Serve Chicken Momos Recipe (Delicious Steamed Chicken Dumplings) along with Peanut Chilli Dipping Sauce for an evening snack.",North East India Recipes,Snack,High Protein Non Vegetarian,30,30,"[2 cups All Purpose Flour (Maida), Salt, to taste, 1-1/2 teaspoon Baking powder, 1 cup Chicken, boiled and minced, 1/2 cup Green peas (Matar), boiled, 1/2 cup Onions, finely chopped, 1 tablespoon Garlic, finely chopped, 1/2 tablespoon Ginger, finely chopped, 1/2 tablespoon Soy sauce, Salt, to taste, 1 Fresh Red chillies, or green chilli, 1/4 tablespoon Whole Black Peppercorns, crushed]","[To prepare Chicken Momos Recipe], prep up with all the ingredients mentioned. Boil the chicken, mince it., Boil the green peas and keep it ready. Grate garlic and ginger or finely chop them., In a mixing bowl, add the refined flour, salt, baking powder and water; and when the flour looks crumbly, knead to a stiff dough, 